In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 


select date, sum(count_a), sum(sum_a) from plproxy.execute_select_nestloop(\$proxy\$ 
    select date,count(est_average_active_users) as count_a, sum(est_average_active_users) as sum_a
    from mu.city_app_w
    where 
        date between '2018-09-01' and '2020-09-05' and est_average_active_users is not null and est_average_active_users <> 0
    group by date
\$proxy\$) tpl (date DATE , count_a BIGINT, sum_a REAL) group by date order by date asc;

EOF

In [0]:

from pyspark.sql import functions as F

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=monthly/date=2020-08-31/").filter('metric_name="est_average_active_users"').filter('value is null or value = 0').show(10000)

# spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/date=2020-09-05/").show(1000)

In [0]:


from applications.db_check_v1.common.utils import get_date_list
from pyspark.sql import functions as F

from datetime import datetime

begin_date = datetime(2018, 9, 1)
end_date = datetime(2020, 9, 5)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


result_list = []
for date_str in DATE_GRANULARITY_MAPPINGLIST["monthly"]:
    data_agg = spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=monthly/date={}/".format(date_str)).filter('metric_name="est_average_active_users" and value is not null and value <> 0').agg(F.count("metric_name").alias("count"),F.sum("value").alias("sum")).collect()
    print "{}|{}|{}".format(date_str,data_agg[0]["count"], data_agg[0]["sum"])
    result_list.append([date_str,data_agg[0]["count"], data_agg[0]["sum"]])
print "**monthly**"*10
print result_list
print "**monthly**"*10


In [0]:

data_agg[0]["count"]

In [0]:
%%sh

aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/

In [0]:
%%sh
